<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Propagate a low-resolution hand-crafted laser pulse through a thin lens.</b>
<br>We initialize the laser pulse from the <b>Pump Off</b> experimental configuration.
</div>

In [ ]:
import rslaser
import matplotlib.pyplot as plt
plt.rcParams['pcolor.shading'] ='auto'
import numpy as np
from pykern.pkcollections import PKDict
from rslaser.optics import crystal
from rslaser.pulse import pulse
from rslaser.optics import drift
from rslaser.optics import lens
import scipy.constants as const
from pykern import pkio
import matplotlib as mpl
import srwlib
from srwlib import srwl
import uti_plot

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline
data_dir = pkio.py_path(rslaser.pkg_resources.resource_filename('rslaser','package_data'))

LP = pulse.LaserPulse(
    PKDict(
        nslice=1,
        photon_e_ev=1.55,
    ), 
    files=PKDict(
        meta=str(data_dir.join('/20220218/meta_data.dat')),
        ccd=str(data_dir.join('/20220218/photon_count_pump_off.txt')),
        wfs=str(data_dir.join('/20220218/phase_pump_off.txt')),
    )
)


In [ ]:
photonss0 = LP.slice[0].n_photons_2d
with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(photonss0.x*1e3, photonss0.y*1e3, np.reshape(photonss0.mesh, (LP.slice_wfr(0).mesh.ny, LP.slice_wfr(0).mesh.nx)), cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Number of Photons')

In [ ]:
e_total = LP.extract_total_2d_elec_fields()
intens0_2d = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
intens0 = intens0_2d.flatten()
wfr0 = LP.slice_wfr(0)

#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(photonss0.x*1e3, photonss0.y*1e3, intens0_2d, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (from SRW wavefront)')

In [ ]:
phase0_2d = LP.extract_total_2d_phase()
phase0 = phase0_2d.flatten()

#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

# plot the phase
with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(photonss0.x*1e3, photonss0.y*1e3, phase0_2d, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_title('Phase (from SRW wavefront)')

In [ ]:
LP.rmsvals()

<div class="alert alert-block alert-info">
<p style="font-size:18px;">Propagate the laser pulse through a Ti:Sapphire crystal and 50 cm drift.</p>
</div>

In [ ]:
# instantiate the crystal with laser pumping on
L_crystal = 0.01   # [m]
num_slices = 1

crystal_params = PKDict(
        n0 = [1.76 for _ in range(num_slices)],
        n2 = [4.0  for _ in range(num_slices)],
        length = L_crystal,
        nslice = num_slices,
#        l_scale = 0.001,    # uncomment when using LCT-based propagators
)
e_crystal = crystal.Crystal(crystal_params)

# instantiate a simple focusing lens
focal_length = 2.   # [m]
e_lens = lens.Lens_srw(focal_length)

# instantiate a drift
L_drift = 0.50  # [m]
e_drift = lens.Drift_srw(L_drift)

lattice = [(e_lens,'default'), 
#           (e_crystal,'abcd_lct'), #'abcd'), 
           (e_crystal,'n0n2_srw'), #'abcd'), 
           (e_lens,'default'), 
           (e_drift,'default')
          ]

In [ ]:
# propagate the pulse through the lattice
LP_final = LP
for iterator in lattice:
    current_elem, prop_type = iterator
    LP_final = current_elem.propagate(LP_final, prop_type)

<div class="alert alert-block alert-info">
<p style="font-size:18px;">Plot the laser pulse after propagating through the crystal and drift.</p>
</div>

In [ ]:
# transmitted pulse - intensity
wfr2=LP_final.slice_wfr(0)

x1 = np.linspace(wfr2.mesh.xStart,wfr2.mesh.xFin,wfr2.mesh.nx)
y1 = np.linspace(wfr2.mesh.yStart,wfr2.mesh.yFin,wfr2.mesh.ny)
x, y = np.meshgrid(x1, y1)

e_total = LP_final.extract_total_2d_elec_fields()
intens2_2d = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
intens2 = intens2_2d.flatten()

#srwlib.srwl_uti_save_intens_ascii(intens2, wfr2.mesh, 'final-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens2,
    [wfr2.mesh.xStart, wfr2.mesh.xFin, wfr2.mesh.nx],
    [wfr2.mesh.yStart, wfr2.mesh.yFin, wfr2.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

In [ ]:
# plot the computed intensity
with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
    plt.pcolormesh(x1*1e3, y1*1e3, intens2_2d, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (after crystal and drift)')

In [ ]:
phase2_2d = LP_final.extract_total_2d_phase()
phase2 = phase2_2d.flatten()

#srwlib.srwl_uti_save_intens_ascii(phase2, wfr2.mesh, 'final-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase2,
    [wfr2.mesh.xStart, wfr2.mesh.xFin, wfr2.mesh.nx],
    [wfr2.mesh.yStart, wfr2.mesh.yFin, wfr2.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()

    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
#    plt.pcolormesh(x*1e3, y*1e3, np.unwrap(phase2_2d), cmap=plt.cm.viridis,shading='auto')
    plt.pcolormesh(x*1e3, y*1e3, phase2_2d, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (after crystal and drift)')

In [ ]:
LP.rmsvals()